# LSH

Ссылки:

* [Numerical vectors: LSH + binary projections](https://colab.research.google.com/drive/181MMOcTnzdMVzJr0pWzqtEG0-BV9BIHH)
* [Texts: LSH + binary projections](http://ethen8181.github.io/machine-learning/recsys/content_based/lsh_text.html)
* [Texts: LSH + MinHash + Bands](https://github.com/mattilyra/LSH/blob/master/examples/Introduction.ipynb)

In [131]:
import pandas as pd

df = pd.read_csv("Tweets.csv",delimiter=',', error_bad_lines=False)

df = df[['text']].dropna()
df.reset_index(drop=True, inplace=True)
df['id'] = np.arange(len(df))

df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 284: expected 4 fields, saw 5\nSkipping line 2398: expected 4 fields, saw 5\nSkipping line 2438: expected 4 fields, saw 6\nSkipping line 3104: expected 4 fields, saw 6\nSkipping line 3618: expected 4 fields, saw 5\nSkipping line 3742: expected 4 fields, saw 7\nSkipping line 4694: expected 4 fields, saw 9\nSkipping line 5272: expected 4 fields, saw 5\nSkipping line 6194: expected 4 fields, saw 5\nSkipping line 7017: expected 4 fields, saw 6\nSkipping line 7786: expected 4 fields, saw 5\nSkipping line 8407: expected 4 fields, saw 5\nSkipping line 8529: expected 4 fields, saw 5\nSkipping line 8820: expected 4 fields, saw 5\nSkipping line 9157: expected 4 fields, saw 7\nSkipping line 9952: expected 4 fields, saw 5\nSkipping line 10203: expec

,text,id
0,Sooo SAD I will miss you here in San Diego!!!,0
1,my boss is bullying me...,1
2,what interview! leave me alone,2
3,http://www.dothebouncy.com/smf - some shameles...,3
4,2am feedings for the baby are fun when he is a...,4


In [132]:
len(df)

21027

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(
    analyzer='char',
    ngram_range=(1, 3),
    min_df=0,
    stop_words='english')
X_tfidf = tfidf.fit_transform(df['text'])
X_tfidf

<21027x24002 sparse matrix of type '<class 'numpy.float64'>'
	with 2580648 stored elements in Compressed Sparse Row format>

In [134]:
import numpy as np

def get_similarity_items(X_tfidf, item_id, topn=5):
    """
    Get the top similar items for a given item id.
    The similarity measure here is based on cosine distance.
    """
    query = X_tfidf[item_id]
    scores = X_tfidf.dot(query.T).toarray().ravel()
    best = np.argpartition(scores, -topn)[-topn:]
    return sorted(zip(best, scores[best]), key=lambda x: -x[1])


similar_items = get_similarity_items(X_tfidf, item_id=15)

# an item is always most similar to itself, in real-world
# scenario we might want to filter itself out from the output
for similar_item, similarity in similar_items:
    print(similar_item, similarity)
    item_description = df.iloc[similar_item]['text']
    print('similar item id: ', similar_item)
    print('cosine similarity: ', similarity)
    print('item description: ', item_description)
    print()

15 1.0000000000000002
similar item id:  15
cosine similarity:  1.0000000000000002
item description:  is cleaning the house for her family who is comming later today..

10072 0.5336534905823506
similar item id:  10072
cosine similarity:  0.5336534905823506
item description:  Cleaning the house

4495 0.48954643197016023
similar item id:  4495
cosine similarity:  0.48954643197016023
item description:   hot weather for the lose. And I have to clean too!  Cleaning in the heat sucks balls.

678 0.45118380147029297
similar item id:  678
cosine similarity:  0.45118380147029297
item description:  Searching my home for a few things to cook them for dinner this evening. It`s mothers day so guess who im eating with

3382 0.4492543819216047
similar item id:  3382
cosine similarity:  0.4492543819216047
item description:  cleaning cleaning cleaning today then working out!  i love not working!



In [135]:
def generate_random_vectors(dim, n_vectors):
    """
    generate random projection vectors
    the dims comes first in the matrix's shape,
    so we can use it for matrix multiplication.
    """
    return np.random.randn(dim, n_vectors)

In [136]:
vocab_size = len(tfidf.get_feature_names())
print('vocabulary size: ', vocab_size)

np.random.seed(0)
n_vectors = 16
random_vectors = generate_random_vectors(vocab_size, n_vectors)
print('dimension: ', random_vectors.shape)
random_vectors

vocabulary size:  24002
dimension:  (24002, 16)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array([[ 1.76405235,  0.40015721,  0.97873798, ...,  0.12167502,
         0.44386323,  0.33367433],
       [ 1.49407907, -0.20515826,  0.3130677 , ...,  1.46935877,
         0.15494743,  0.37816252],
       [-0.88778575, -1.98079647, -0.34791215, ..., -0.4380743 ,
        -1.25279536,  0.77749036],
       ...,
       [-1.31821475,  0.36817968,  0.53415198, ...,  1.60818267,
         0.57866133, -1.7089848 ],
       [-0.83863877,  0.61162805,  0.83759737, ...,  0.8996262 ,
         0.04076996, -2.66958722],
       [-0.28932216,  0.7880676 , -0.01545824, ..., -1.54590442,
         0.54839386,  0.87119482]])

In [138]:
data_point = X_tfidf[0]

# True if positive sign; False if negative sign
bin_indices_bits = data_point.dot(random_vectors) >= 0
print('dimension: ', bin_indices_bits.shape)
bin_indices_bits

dimension:  (1, 16)


array([[ True,  True, False,  True, False, False,  True,  True, False,
        False, False, False,  True,  True,  True, False]])

In [139]:
bin_indices_bits = data_point.dot(random_vectors) >= 0

# https://wiki.python.org/moin/BitwiseOperators
# x << y is the same as multiplying x by 2 ** y
powers_of_two = 1 << np.arange(n_vectors - 1, -1, step=-1)
print(powers_of_two)

# final integer representation of individual bins
bin_indices = bin_indices_bits.dot(powers_of_two)
print(bin_indices)

[32768 16384  8192  4096  2048  1024   512   256   128    64    32    16
     8     4     2     1]
[54030]


In [140]:
# we can do it for the entire corpus
bin_indices_bits = X_tfidf.dot(random_vectors) >= 0
print(bin_indices_bits.shape)
bin_indices = bin_indices_bits.dot(powers_of_two)
bin_indices.shape

(21027, 16)


(21027,)

In [141]:
from collections import defaultdict


def train_lsh(X_tfidf, n_vectors, seed=None):    
    if seed is not None:
        np.random.seed(seed)

    dim = X_tfidf.shape[1]
    random_vectors = generate_random_vectors(dim, n_vectors)  

    # partition data points into bins,
    # and encode bin index bits into integers
    bin_indices_bits = X_tfidf.dot(random_vectors) >= 0
    powers_of_two = 1 << np.arange(n_vectors - 1, -1, step=-1)
    bin_indices = bin_indices_bits.dot(powers_of_two)

    # update `table` so that `table[i]` is the list of document ids with bin index equal to i
    table = defaultdict(list)
    for idx, bin_index in enumerate(bin_indices):
        table[bin_index].append(idx)
    
    # note that we're storing the bin_indices here
    # so we can do some ad-hoc checking with it,
    # this isn't actually required
    model = {'table': table,
             'random_vectors': random_vectors,
             'bin_indices': bin_indices,
             'bin_indices_bits': bin_indices_bits}
    return model


# train the model
n_vectors = 16
model = train_lsh(X_tfidf, n_vectors, seed=143)

In [142]:
# comparison
similar_item_ids = [similar_item for similar_item, _ in similar_items]
bits1 = model['bin_indices_bits'][similar_item_ids[0]]
bits2 = model['bin_indices_bits'][similar_item_ids[1]]

print('bits 1: ', bits1)
print('bits 2: ', bits2)
print('Number of agreed bins: ', np.sum(bits1 == bits2))

bits 1:  [False False  True False  True  True False False False  True  True False
 False False False False]
bits 2:  [False False  True False  True  True  True False False  True False False
 False  True False False]
Number of agreed bins:  13


In [143]:
from itertools import combinations


def search_nearby_bins(query_bin_bits, table, search_radius=3, candidate_set=None):
    """
    For a given query vector and trained LSH model's table
    return all candidate neighbors with the specified search radius.
    
    Example
    -------
    model = train_lsh(X_tfidf, n_vectors=16, seed=143)
    query = model['bin_index_bits'][0]  # vector for the first document
    candidates = search_nearby_bins(query, model['table'])
    """
    if candidate_set is None:
        candidate_set = set()

    n_vectors = query_bin_bits.shape[0]
    powers_of_two = 1 << np.arange(n_vectors - 1, -1, step=-1)

    for different_bits in combinations(range(n_vectors), search_radius):
        # flip the bits (n_1, n_2, ..., n_r) of the query bin to produce a new bit vector
        index = list(different_bits)
        alternate_bits = query_bin_bits.copy()
        alternate_bits[index] = np.logical_not(alternate_bits[index])

        # convert the new bit vector to an integer index
        nearby_bin = alternate_bits.dot(powers_of_two)

        # fetch the list of documents belonging to
        # the bin indexed by the new bit vector,
        # then add those documents to candidate_set;
        # make sure that the bin exists in the table
        if nearby_bin in table:
            candidate_set.update(table[nearby_bin])

    return candidate_set

In [144]:
from sklearn.metrics.pairwise import pairwise_distances


def get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=3):
    table = model['table']
    random_vectors = model['random_vectors']

    # compute bin index for the query vector, in bit representation.
    bin_index_bits = np.ravel(query_vector.dot(random_vectors) >= 0)

    # search nearby bins and collect candidates
    candidate_set = set()
    for search_radius in range(max_search_radius + 1):
        candidate_set = search_nearby_bins(bin_index_bits, table, search_radius, candidate_set)

    # sort candidates by their true distances from the query
    candidate_list = list(candidate_set)
    candidates = X_tfidf[candidate_list]
    distance = pairwise_distances(candidates, query_vector, metric='cosine').flatten()
    
    distance_col = 'distance'
    nearest_neighbors = pd.DataFrame({
        'id': candidate_list, distance_col: distance
    }).sort_values(distance_col).reset_index(drop=True)
    return nearest_neighbors

In [145]:
print('original similar items:\n' + str(similar_items))

item_id = 15
query_vector = X_tfidf[item_id]
nearest_neighbors = get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=5)
print('dimension: ', nearest_neighbors.shape)
nearest_neighbors.head()

original similar items:
[(15, 1.0000000000000002), (10072, 0.5336534905823506), (4495, 0.48954643197016023), (678, 0.45118380147029297), (3382, 0.4492543819216047)]
dimension:  (3292, 2)


,id,distance
0,15,0.000000
1,10072,0.466347
2,4495,0.510454
3,3382,0.550746
4,12193,0.552188


In [146]:
nearest_neighbors = get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=10)
print('dimension: ', nearest_neighbors.shape)
nearest_neighbors.head()

dimension:  (20087, 2)


,id,distance
0,15,0.000000
1,10072,0.466347
2,4495,0.510454
3,678,0.548816
4,3382,0.550746


In [147]:
# we can perform a join with the original table to get the description
# for sanity checking purpose
nearest_neighbors.head().merge(df, on='id', how='inner')

,id,distance,text
0,15,0.000000,is cleaning the house for her family who is co...
1,10072,0.466347,Cleaning the house
2,4495,0.510454,hot weather for the lose. And I have to clean...
3,678,0.548816,Searching my home for a few things to cook the...
4,3382,0.550746,cleaning cleaning cleaning today then working ...
